In [31]:
a = [[9] * 4 for _ in range(3)]
a

[[9, 9, 9, 9], [9, 9, 9, 9], [9, 9, 9, 9]]

In [ ]:
class Solution {
    int dp[101][101][27][11];
public:
    int getLengthOfOptimalCompression(string s, int K) 
    {
        int n = s.size();
        s = "#"+s;

        for (int i=0; i<=n; i++)
            for (int k=0; k<=K; k++)
                for (int ch=0; ch<=26; ch++)
                    for (int num=0; num<=10; num++)
                        dp[i][k][ch][num] = INT_MAX;
        # dp[i][k][ch][num]: the optimal solution for s[1:i]
        #  with k digits removed, last letter as ch, the consecitive number of ch as num

        dp[0][0][26][0] = 0;

        for (int i=0; i<n; i++)
            for (int k=0; k<=min(K, i); k++)
                for (int ch = 0; ch <= 26; ch++)
                    for (int num = 0; num <=10; num++)
                    {
                        int cur = dp[i][k][ch][num];
                        if (cur==INT_MAX) continue;

                        // delete s[i+1]
                        dp[i+1][k+1][ch][num] = min(dp[i+1][k+1][ch][num], cur);

                        // keep s[i+1]
                        if (s[i+1]-'a' == ch)
                        {
                            int add;
                            if (num==1) add = 1;  // a -> a2
                            else if (num>=2 && num<=8) add = 0; // a3->a4
                            else if (num==9) add = 1; // a9->a10;
                            else if (num==10) add = 0; // a10->a11;
                            dp[i+1][k][ch][min(num+1, 10)] = min(dp[i+1][k][ch][min(num+1, 10)], cur+add);
                        }
                        else
                        {
                            dp[i+1][k][s[i+1]-'a'][1] = min(dp[i+1][k][s[i+1]-'a'][1], cur+1);
                        }
                    }
        
        int ret = INT_MAX;
        for (int ch = 0; ch <= 26; ch++)
            for (int num = 0; num <=10; num++)                    
                ret = min(ret, dp[n][K][ch][num]);
        
        return ret;
    }
};


In [142]:
from collections import defaultdict

class Solution:
    def getLengthOfOptimalCompression(self, s: str, k: int) -> int:
        if len(set(s)) == 1: # 如果 s 等于100个相同的字母，那么总长度就是 4，'a100'
            return 4
        
        n = len(s)
        # dp[i][j][ch][num]:
        # s[:i] 一共 remove 了 j 次，最后一个字母为 ch，连续出现了 num 次，最短的 running lenth的长度
        dp = [[[[float('inf')] * 11 for _ in range(27)]  for _ in range(101)] for _ in range(101)] 
        dp[0][0][26][0] = 0 # s长度为 0的时候，remove 0次，最短的长度为 0
        
        s = [ord(x) - ord('a') for x in s]
        s = [-1] + s
        for i in range(n):
            for j in range(100):
                for p in range(26 + 1):
                    for q in range(11):
                        cur = dp[i][j][p][q]
                        if cur == float('inf'): # 如果当前状态不合法，continue
                            continue

                        # 删除掉 s[i]
                        dp[i+1][k+1][p][q] = min(dp[i+1][k+1][p][q], cur)
                        
                        # 保留 s[i+1], 如果s[i+1] 不等于 s[i]
                        if p != s[i+1]:
                            dp[i+1][k][s[i+1]][1] = min(dp[i+1][k][s[i+1]][1], cur + 1)
                        # 保留 s[i+1], 如果s[i+1] 等于 s[i], 如过之前出现了 9次，需要 +1
                        else:
                            # 0 -> 1, 9 -> 10
                            add = 1 if q == 1 or q == 9 else 0 
                            dp[i+1][k][p][min(q+1, 10)] = min(dp[i+1][k][p][min(q+1, 10)], cur + add)
                        
                        
        res = float('inf')
        for i in range(26+1):
            for j in range(11):
                res = min(res, dp[n][k][i][j])
        return res

In [182]:
from collections import defaultdict
from functools import lru_cache

class Solution:
    def getLengthOfOptimalCompression(self, s: str, k: int) -> int:
        @lru_cache(None)
        def dp(start, k):
            # 从字符串的第start位置开始，可以最多删除 k 个元素
            # if (start, k) in memo:
            #     return memo[(start, k)]
            # n-start <= k, 可以将sub_str中的字符全部移除
            if start == n or n - start <= k: 
                return 0
            
            ans = float('inf') # 记录 [start:n]最多删除k个元素，compress str 的最小长度是多少 
            count = defaultdict(int) # 记录每个元素出现的频率
            most_freq = 0 # s[start:n] 中元素最多出现的个数是多少
            for j in range(start, n):
                c = s[j]
                count[c] += 1
                most_freq = max(most_freq, count[c]) # 更新元素最多出现的次数
                
                # 出现 [次数最多的字母] 在压缩后 running str 的长度
                compressed_len = 1 + (len(str(most_freq)) if most_freq > 1 else 0)
                # 使用 k 移除掉所有 非 出现次数最多的字母：
                # j-start+1: sub_str 的长度
                # most_freq: sub_str 中出现字母最多的次数
                if k - (j - start + 1 - most_freq) >= 0: 
                    ans = min(ans, compressed_len + dp(j+1, k - (j - start + 1 - most_freq)))
            print(s[:start+1], k, ans)
            # memo[(start, k)] = ans
            return ans
        
        n = len(s)
        # memo = {}
        return dp(0, k)

In [183]:
solution = Solution()
solution.getLengthOfOptimalCompression(s = "abacaadaa", k = 2)

abacaad 2 1
abacaada 1 1
abacaa 2 2
abaca 2 2
abacaadaa 0 1
abacaad 1 2
abacaada 0 2
abacaa 1 2
abac 2 2
abaca 1 2
aba 2 2
abacaad 0 3
abacaa 0 4
abac 1 3
abaca 0 5
ab 2 3
aba 1 4
a 2 4


4

In [ ]:
abacaadaa
abaca2da2